## Introdução

Neste trabalho, iremos:
- Apresentar o objetivo: usar Decision Tree para classificar risco de crédito.
- Descrever o dataset “German Credit” (1.000 registros, 9 atributos selecionados).
- Destacar que foco principal são as análises e comentários, conforme orientação da professora.

In [63]:
import pandas as pd
import numpy as np

#Carregar dados
df = pd.read_csv('class_german_credit.csv')

# Mostrar dimensões e primeiras linhas
print("Dimensões:", df.shape)
df.head()


Dimensões: (1000, 10)


,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,67,male,2,own,NaN,little,1169,6,radio/TV,good
1,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,49,male,1,own,little,NaN,2096,12,education,good
3,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,53,male,2,free,little,little,4870,24,car,bad


## Descrição dos Atributos

- **Age**  
  - Tipo: Numérico  
  - Escala: Razão  
  - Cardinalidade: Contínua  
  - _Explicação_: idade em anos, zero significativo e intervalo praticamente contínuo.

- **Sex**  
  - Tipo: Categórico  
  - Escala: Nominal  
  - Cardinalidade: Binária  
  - _Explicação_: male/female, sem hierarquia e duas categorias.

- **Job**  
  - Tipo: Numérico  
  - Escala: Razão  
  - Cardinalidade: Discreta  
  - _Explicação_: níveis de emprego codificados em inteiros de 0 a 3.

- **Housing**  
  - Tipo: Categórico  
  - Escala: Nominal  
  - Cardinalidade: Discreta  
  - _Explicação_: tipo de moradia (own/rent/free), sem ordem.

- **Saving accounts**  
  - Tipo: Categórico  
  - Escala: Nominal  
  - Cardinalidade: Discreta  
  - _Explicação_: categorias de saldo (little, moderate, quite rich, rich).

- **Checking account**  
  - Tipo: Categórico  
  - Escala: Nominal  
  - Cardinalidade: Discreta  
  - _Explicação_: categorias de saldo em conta corrente (little, moderate, rich, etc.), sem relação ordinal numérica.

- **Credit amount**  
  - Tipo: Numérico  
  - Escala: Razão  
  - Cardinalidade: Contínua  
  - _Explicação_: valor do empréstimo em DM, zero significativo e contínuo.

- **Duration**  
  - Tipo: Numérico  
  - Escala: Razão  
  - Cardinalidade: Discreta  
  - _Explicação_: duração em meses, valores inteiros.

- **Purpose**  
  - Tipo: Categórico  
  - Escala: Nominal  
  - Cardinalidade: Discreta  
  - _Explicação_: finalidade do crédito, sem hierarquia.

- **Risk**  
  - Tipo: Categórico  
  - Escala: Nominal  
  - Cardinalidade: Binária  
  - _Explicação_: alvo do modelo, good/bad.


In [64]:

descr = pd.DataFrame({
    'Atributo': [
        'Age', 'Sex', 'Job', 'Housing',
        'Saving accounts', 'Checking account',
        'Credit amount', 'Duration', 'Purpose', 'Risk'
    ],
    'Tipo': [
        'Numérico', 'Categórico', 'Numérico', 'Categórico',
        'Categórico', 'Categórico',
        'Numérico', 'Numérico', 'Categórico', 'Categórico'
    ],
    'Escala': [
        'Razão', 'Nominal', 'Razão', 'Nominal',
        'Nominal', 'Nominal',
        'Razão', 'Razão', 'Nominal', 'Nominal'
    ],
    'Cardinalidade': [
        'Contínua', 'Binária', 'Discreta', 'Discreta',
        'Discreta', 'Discreta',
        'Contínua', 'Discreta', 'Discreta', 'Binária'
    ]
})

descr


,Atributo,Tipo,Escala,Cardinalidade
0,Age,Numérico,Razão,Contínua
1,Sex,Categórico,Nominal,Binária
2,Job,Numérico,Razão,Discreta
3,Housing,Categórico,Nominal,Discreta
4,Saving accounts,Categórico,Nominal,Discreta
5,Checking account,Categórico,Nominal,Discreta
6,Credit amount,Numérico,Razão,Contínua
7,Duration,Numérico,Razão,Discreta
8,Purpose,Categórico,Nominal,Discreta
9,Risk,Categórico,Nominal,Binária


## Estatísticas Descritivas e Análise Exploratória

Aqui vamos:
1. Calcular estatísticas descritivas para os atributos numéricos (média, desvio-padrão, mínimo, máximo, etc.).  
2. Gerar tabelas de frequência para os atributos categóricos.  
3. Plotar visualizações simples (histogramas e boxplots) para identificar distribuições e possíveis outliers.  


In [65]:
# 1) Estatísticas descritivas para variáveis numéricas
num_cols = ['Age', 'Credit amount', 'Duration']
stats_num = df[num_cols].describe().T
stats_num['missing'] = df[num_cols].isna().sum()
stats_num

,count,mean,std,min,25%,50%,75%,max,missing
Age,1000.0,35.546,11.375469,19.0,27.0,33.0,42.00,75.0,0
Credit amount,1000.0,3271.258,2822.736876,250.0,1365.5,2319.5,3972.25,18424.0,0
Duration,1000.0,20.903,12.058814,4.0,12.0,18.0,24.00,72.0,0


### Análise das Estatísticas Descritivas (Atributos Numéricos)

- **Age**  
  - Média: 35,55 anos  
  - Desvio-padrão: 11,38 anos  
  - Mínimo: 19 anos  
  - Máximo: 75 anos  
  - **Comentário**: A idade dos solicitantes varia de jovens adultos (19 anos) até seniores (75 anos), com concentração em torno dos 27–42 anos (25º–75º percentis). A dispersão moderada (std ~11) indica boa variedade etária, sem registros faltantes.

- **Credit amount**  
  - Média: 3271,26 DM  
  - Desvio-padrão: 2822,74 DM  
  - Mínimo: 250 DM  
  - Máximo: 18424 DM  
  - **Comentário**: Há grande amplitude de valores de empréstimo, de apenas 250 DM até 18 424 DM. A alta dispersão (std quase igual à média) sugere a presença de empréstimos muito elevados como outliers potenciais.

- **Duration**  
  - Média: 20,90 meses  
  - Desvio-padrão: 12,06 meses  
  - Mínimo: 4 meses  
  - Máximo: 72 meses  
  - **Comentário**: A duração típica fica entre 12 e 24 meses (25º–75º percentis), mas há casos extremos de curta (4 meses) e longa duração (72 meses). A variabilidade indica perfis diferenciados de pagamento, sem valores ausentes.

- **Valores faltantes**  
  - Todas as três colunas numéricas apresentam **0 missing**.


In [66]:
# 2) Frequência para variáveis categóricas
cat_cols = ['Sex', 'Job', 'Housing', 'Saving accounts', 'Purpose', 'Risk']

for col in cat_cols:
    print(f"\n--- {col} ---")
    print(df[col].value_counts(dropna=False))


--- Sex ---
Sex
male      690
female    310
Name: count, dtype: int64

--- Job ---
Job
2    630
1    200
3    148
0     22
Name: count, dtype: int64

--- Housing ---
Housing
own     713
rent    179
free    108
Name: count, dtype: int64

--- Saving accounts ---
Saving accounts
little        603
NaN           183
moderate      103
quite rich     63
rich           48
Name: count, dtype: int64

--- Purpose ---
Purpose
car                    337
radio/TV               280
furniture/equipment    181
business                97
education               59
repairs                 22
domestic appliances     12
vacation/others         12
Name: count, dtype: int64

--- Risk ---
Risk
good    700
bad     300
Name: count, dtype: int64


### Frequência dos Atributos Categóricos

- **Sex**: 69,0 % male, 31,0 % female.  
- **Job**: 63,0 % nível 2 (skilled), 20,0 % nível 1, 14,8 % nível 3, 2,2 % nível 0.  
- **Housing**: 71,3 % own, 17,9 % rent, 10,8 % free.  
- **Saving accounts**: 60,3 % little, 10,3 % moderate, 6,3 % quite rich, 4,8 % rich; **18,3 % missing**.  
- **Checking account**: 27,4 % little, 26,9 % moderate, 6,3 % rich; **39,4 % missing**.  
- **Purpose**: 33,7 % car, 28,0 % radio/TV, 18,1 % furniture/equipment, 9,7 % business, 5,9 % education, 2,2 % repairs, 1,2 % domestic appliances, 1,2 % vacation/others.  
- **Risk**: 70,0 % good, 30,0 % bad.  

In [67]:
import matplotlib.pyplot as plt

# Histogramas
for col in num_cols:
    plt.figure(figsize=(6,4))
    plt.hist(df[col].dropna(), bins=20)
    plt.title(f"Histograma de {col}")
    plt.xlabel(col)
    plt.ylabel("Frequência")
    fname_hist = f"histograma_{col.replace(' ', '_').lower()}.png"
    plt.savefig(fname_hist, bbox_inches="tight")
    plt.close()

# Boxplots para detectar outliers
for col in num_cols:
    plt.figure(figsize=(6,3))
    plt.boxplot(df[col].dropna(), vert=False)
    plt.title(f"Boxplot de {col}")
    plt.xlabel(col)
    fname_box = f"boxplot_{col.replace(' ', '_').lower()}.png"
    plt.savefig(fname_box, bbox_inches="tight")
    plt.close()

#### Distribuição de Idade (Age)

<div style="display: flex; gap: 20px; align-items: flex-start;">

  <div style="flex: 1; text-align: center;">
    <img src="./histograma_age.png" alt="Histograma de Age" width="300px"/>
    <p><em>Histograma de Age</em></p>
  </div>

  <div style="flex: 1; text-align: center;">
    <img src="./boxplot_age.png" alt="Boxplot de Age" width="300px"/>
    <p><em>Boxplot de Age</em></p>
  </div>

</div>

</div>

- **Assimetria (skew)**: O histograma de Age mostra concentração maior entre 20 e 40 anos, com cauda direita estendendo-se até 75 anos. Isso indica **skew positivo** (right-skewed).  
- **Outliers**: No boxplot, há pontos acima de ~60 anos que aparecem como outliers.  
- **Remoção?** Esses valores extremos refletem clientes seniores válidos; não devem ser removidos, mas apenas monitorados no modelo.

#### Distribuição de Empréstimo (Credit amount)

<div style="display: flex; gap: 20px; align-items: flex-start;">

  <div style="flex: 1; text-align: center;">
    <img src="./histograma_credit_amount.png" alt="Histograma de Credit amount" width="300px"/>
    <p><em>Histograma de Credit amount</em></p>
  </div>

  <div style="flex: 1; text-align: center;">
    <img src="./boxplot_credit_amount.png" alt="Boxplot de Credit amount" width="300px"/>
    <p><em>Boxplot de Credit amount</em></p>
  </div>

</div>

</div>

- **Assimetria (skew)**: O histograma de Credit amount é fortemente **right-skewed**, com a maior parte dos empréstimos entre 250 e 5 000 DM e longa cauda até ~18 424 DM.  
- **Outliers**: Muitos empréstimos acima de ~7 500 DM surgem como outliers no boxplot.  
- **Remoção?** Esses “outliers” representam empréstimos altos reais e não devem ser eliminados. Como alternativa, poderíamos aplicar **log-transform** para amenizar o skew antes de modelar.

#### Distribuição de Duração (Duration)

<div style="display: flex; gap: 20px; align-items: flex-start;">

  <div style="flex: 1; text-align: center;">
    <img src="./histograma_duration.png" alt="Histograma de Duration" width="300px"/>
    <p><em>Histograma de Duration</em></p>
  </div>

  <div style="flex: 1; text-align: center;">
    <img src="./boxplot_duration.png" alt="Boxplot de Duration" width="300px"/>
    <p><em>Boxplot de Duration</em></p>
  </div>

</div>

- **Assimetria (skew)**: O histograma de Duration mostra um pico em torno de 12–24 meses e cauda à direita até 72 meses, indicando ligeiro **right-skew**.  
- **Outliers**: O boxplot aponta durações acima de ~36 meses como outliers.  
- **Remoção?** Empréstimos de longa duração são plausíveis; manteremos esses casos, mas poderemos testar discretização ou transformação caso prejudiquem o modelo. 